# IMPORTACIONES

In [1]:
import pandas as pd
import numpy as np

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
df_dia = pd.read_csv("./datasets/productos_dia.csv")
df_dia.head()

,Unnamed: 0,url,supermarket,name,description,product_id
0,2362486,https://www.dia.es/compra-online/cuidado-del-h...,dia-es,MISTOL lavavajillas mano original botella 900 ml,NaN,9e2fda45e63337294f2a604178823026
1,2362487,https://www.dia.es/compra-online/cuidado-del-h...,dia-es,DIA lavavajillas máquina premium todo en uno e...,NaN,244994c762813d38881cb118742152a2
2,2362488,https://www.dia.es/compra-online/cuidado-del-h...,dia-es,FAIRY lavavajillas máquina platinum azul en cá...,NaN,8a5398b602e44b93a0c0a4a7b9d22b84
3,2362489,https://www.dia.es/compra-online/cuidado-del-h...,dia-es,FAIRY lavavajillas máquina platinum plus limón...,NaN,2cd7ea87b1963adb51d8e1295f6c14d3
4,2362501,https://www.dia.es/compra-online/cuidado-del-h...,dia-es,FAIRY lavavajillas mano concentrado aloe vera ...,NaN,4abac3d30922f81012d7ae0c2cd955f2


# Repaso II: Productos_dia y Precio_productos

Imagina que eres un analista de datos para el supermercado Día, y te proporcionan los datos de los productos y sus precios.

Para ello podemos hacer una análisis de los datos para obtener información relevante, sobre:

1. Identificar la categoría y subcategoría de cada producto

In [6]:
# Identificar categoría y subcategoría

df_dia[["categoria","subcategoria","otros"]] = df_dia["url"].str.split("/", expand= True).get([4,5,6])
df_dia.head(3)

,Unnamed: 0,url,supermarket,name,description,product_id,categoria,subcategoria,otros
0,2362486,https://www.dia.es/compra-online/cuidado-del-h...,dia-es,MISTOL lavavajillas mano original botella 900 ml,NaN,9e2fda45e63337294f2a604178823026,cuidado-del-hogar,lavavajillas,p
1,2362487,https://www.dia.es/compra-online/cuidado-del-h...,dia-es,DIA lavavajillas máquina premium todo en uno e...,NaN,244994c762813d38881cb118742152a2,cuidado-del-hogar,lavavajillas,p
2,2362488,https://www.dia.es/compra-online/cuidado-del-h...,dia-es,FAIRY lavavajillas máquina platinum azul en cá...,NaN,8a5398b602e44b93a0c0a4a7b9d22b84,cuidado-del-hogar,lavavajillas,p


In [20]:
df_dia['categoria'].unique()

array(['cuidado-del-hogar', 'cuidado-del-hogar-de-limpieza', 'mascotas',
       'platos-preparados', 'al-dia', 'despensa', 'bebidas', 'bodega',
       'bebe', 'congelados', 'cuidado-personal', 'eco', 'caducidad',
       'peso', 'frescos', 'dulces-de-navidad', 'bulto-mixto',
       'desayunos-y-dulces', 'cuidado-corporal', 'conservas'],
      dtype=object)

In [21]:
df_filtrado = df_dia[df_dia["categoria"].isin(["clasificacion-de-categorias"])]
df_filtrado

,Unnamed: 0,url,supermarket,name,description,product_id,categoria,subcategoria,otros


In [ ]:
df_dia.shape

(11494, 9)

In [16]:
df_filtrado.shape

(3, 9)

In [17]:
condicion = df_dia['categoria'] == 'clasificacion-de-categorias'
df_dia.loc[condicion, "categoria"] = df_dia.loc[condicion, "subcategoria"]
df_dia.loc[condicion, "subcategoria"] = df_dia.loc[condicion, "otros"]

2. ¿Qué categoría es la que vende más productos?

In [36]:
# 2. ¿Qué categoría es la que vende más productos?
df_ventas = df_dia.groupby("categoria")["name"].count().reset_index().sort_values(by="name", ascending=False)
df_ventas.head(1)

,categoria,name
13,despensa,3726


In [ ]:
#cómo sacar df del máximo con el nombre ce la cateoría como arriba?????
df_ventas = df_dia.groupby("categoria")["name"].count().max()
df_ventas

np.int64(3726)

In [23]:
df_dia["categoria"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 11494 entries, 0 to 11493
Series name: categoria
Non-Null Count  Dtype 
--------------  ----- 
11494 non-null  object
dtypes: object(1)
memory usage: 89.9+ KB


3. Dentro de cada categoría, ¿qué subcategorías son más populares?

In [ ]:
# Dentro de cada categoría, ¿qué subcategorías son más populares?
df_subcategorias = df_dia.groupby(["categoria", "subcategoria"])["name"].count().reset_index().sort_values(by="name", ascending=False)
df_subcategorias.head(10)

In [ ]:
df_cat_subcat = df_dia.groupby(['categoria','subcategoria'])['name'].count().reset_index()
df_cat_subcat = df_cat_subcat.sort_values(by=["categoria", "name"], ascending=[True, False])
df_cat_subcat

In [ ]:
top3_por_categoria = df_cat_subcat.groupby('categoria').head(3)
top3_por_categoria

4. ¿Qué categoría es la que tiene los precios más altos? ¿y los mínimos? ¿Cuál es su media?

In [ ]:
df_precios = pd.read_csv("./datasets/precios_productos.csv", index_col = 0, decimal=",", thousands=".")

In [60]:
df_precios.columns

Index(['price', 'reference_price', 'product_id', 'reference_unit',
       'insert_date'],
      dtype='object')

In [61]:
df_dia.columns

Index(['Unnamed: 0', 'url', 'supermarket', 'name', 'description', 'product_id',
       'categoria', 'subcategoria', 'otros'],
      dtype='object')

In [ ]:
merge_inner = df_dia.merge(df_precios, left_on = 'product_id', right_on = 'product_id')
#.merge(): Combina dos DataFrames basándose en una columna común (similar a un JOIN en SQL).
#left_on='ID': Columna del DataFrame de la izquierda (df_años) que se usará para hacer la unión.
#right_on='id_': Columna del DataFrame de la derecha (df) que se usará para hacer la unión.
#how='inner' (por defecto): Solo mantiene las filas donde hay coincidencia en ambos DataFrames.
merge_inner.head(2)

In [63]:
print(df_dia.shape)
print(df_precios.shape)
print(merge_inner.shape)

(11494, 9)
(1000000, 5)
(1264512, 13)


In [69]:
media = merge_inner.groupby(['categoria'])['price'].mean().reset_index().sort_values(by="price", ascending=False).round(2)
media.head(1)

,categoria,price
3,bodega,6.33


In [70]:
maximo = merge_inner.groupby(['categoria'])['price'].max().reset_index().sort_values(by="price", ascending=False).round(2)
maximo.head(1)

,categoria,price
16,frescos,289.0


In [71]:
minimo = merge_inner.groupby(['categoria'])['price'].min().reset_index().sort_values(by="price", ascending=True).round(2)
minimo.head(1)

,categoria,price
14,dulces-de-navidad,0.08


## Análisis de precios y productos
1. ¿Existen productos con precios fuera de lo común (outliers)?

2. ¿Cuántos productos tienen un precio superior al precio de referencia?

3. ¿Qué categorías de productos tienen la mayor y menor variación de precios?

4. ¿Cómo varían los precios de los productos a lo largo del tiempo?

## Análisis de productos
1. ¿Qué productos tienen más registros de precios?

2. ¿Cuántos productos distintos hay en el dataset?

3. ¿Existen productos en el dataset de precios que no tienen información en el dataset de productos?

## Relación entre precios y unidades de medida
1. ¿Cuál es el precio medio por unidad de referencia de cada producto?

Recuerda que para realizar este análisis, deberás unir los dos conjuntos de datos de la forma más eficiente. Y realizar un exploración y limpieza de los datos

In [ ]:
.

# Los datos
Para este repaso usaremos dos conjuntos de datos, estos son:

productos_dia.csv: en el que podremos encontrar una serie de productos, y sus precios. Las columnas que hay son:

- url: Es el enlace URL asociado al producto en el sitio web de un supermercado específico. Por ejemplo, el primer registro tiene una URL que probablemente lleva a la página del producto en el sitio web de DIA.

- supermarket: Es el nombre o identificador del supermercado. Por ejemplo, "dia-es" puede referirse al supermercado DIA.

- name: Es el nombre del producto. Por ejemplo, el primer registro describe un producto llamado "MISTOL lavavajillas mano original botella 900 ml".

- description: Proporciona una descripción detallada del producto. Por ejemplo, el segundo registro describe un producto llamado "DIA lavavajillas máquina premium todo en uno e...".

- product_id: Es un identificador único para cada producto. Sin embargo, en estos casos específicos, parece haber valores faltantes (NaN) en esta columna. Cada valor único en esta columna representa un producto diferente.

precios_productos.csv: en el que encontramos todos los productos que han comprando y vendido en el supermercado desde el 2021. Las columnas que tenemos son:

- price: Este es el precio del producto en la moneda especificada. Por ejemplo, el primer registro tiene un precio de 0,7 en alguna moneda no especificada.

- reference_price: Se refiere al precio de referencia del producto. Podría ser el precio original, el precio de venta sugerido u otro precio relevante asociado al producto. Por ejemplo, el segundo registro tiene un precio de referencia de 10,63.

- product_id: Es un identificador único para cada producto. Cada valor único en esta columna representa un producto diferente. Por ejemplo, "9b1d806ddaed2a3b4c842c361e9171a8" es el ID de un producto.

- reference_unit: Indica la unidad de medida utilizada para el precio de referencia del producto. Por ejemplo, "ud" podría representar unidades (cada unidad), "kg" kilogramos, o "l" litros.

- insert_date: Es la fecha en la que se insertó este registro en el DataFrame. Por ejemplo, el primer registro fue insertado el 8 de junio de 2022 a las 00:00:00.